In [ ]:
from MTSP_DS_MILP_Solver import MTSP_DS_MILP_Solver

from core.Location import rand_location
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
n = 8
Dn = 2
Kn = 2

n_configuration = 3
n_trial = 3
num_of_ds = [0,1,2]

In [ ]:
def randomLocations(num_of_locations):
    locs = []
    for _ in range(num_of_locations):
        locs.append(rand_location())
    return locs

In [ ]:
milp_fitness_average = []
for i in range(n_configuration):
    milp_fitness = []
    customer_location = randomLocations(n)
    print("customers: ", customer_location)
    for ds_number in num_of_ds:
        milp_solutions = []
        for _ in range(n_trial):
            while True:  
                ds_setup = randomLocations(ds_number)
                print("ds: ", ds_setup)
                setup = customer_location + ds_setup
                try:
                    milp_solver = MTSP_DS_MILP_Solver(n, ds_number, Kn = 2, eps= 80, custom_locations=setup)
                    milp_solver.solve()
                    milp_solutions.append(milp_solver.getSolution())
                    milp_solver.plot_tours()
                    break 
                except (Exception, TimeoutError) as e:
                    print(f"Errore durante l'esecuzione: {e}. Ritentando con un nuovo setup.")
                    continue
        milp_fitness.append(np.mean(milp_solutions))
        
    print(milp_fitness)
    milp_fitness_average.append(milp_fitness)
print(milp_fitness_average)

In [ ]:
def get_improving_percentage(value, baseline):
    return (baseline - value) / baseline * 100

In [ ]:
fitness_percentages = []
for i in range(n_configuration):
    improving_percentage = []
    for j in range(len(milp_fitness_average[i])):
        improving_percentage.append(get_improving_percentage(milp_fitness_average[i][j], milp_fitness_average[i][0]))
    fitness_percentages.append(improving_percentage)

transposed = zip(*fitness_percentages)
average_fitness_improvement_percentage = [sum(group) / len(group) for group in transposed]
print("Average fitness improvement vs Baseline: \n", average_fitness_improvement_percentage) 

In [ ]:
def plot_sensitivity_analysis(values):
    plt.figure(figsize=(10, 6))
    plt.plot(values, marker='o', linestyle='-', color='b')
    plt.axhline(0, color='gray', linestyle='--')

    plt.xlabel('m (Drone Station number)')
    plt.ylabel('Fitness average improvement vs. Baseline (0 Drone Stations) (%)')
    plt.title('Sensitivity Analysis of parameter m')
    plt.grid(True)
    plt.show()

plot_sensitivity_analysis(average_fitness_improvement_percentage)